In [3]:
import pandas as pd
import geopandas as gpd

RNAL = pd.read_csv("../python/final/porto.csv")
RNAL["NrRNAL"] = RNAL["NrRNAL"].astype(str)
RNAL.head()

,Unnamed: 0,X,Y,OBJECTID,NrRNAL,Denominacao,DataRegisto,DataAberturaPublico,Modalidade,NrUtentes,Email,Endereco,CodigoPostal,Freguesia,Concelho,Distrito
0,100,-8.630320,41.158010,185476533,519,Koolhouse Porto,2014/12/05 10:19:59+00,2014/12/05 00:00:00+00,EstabelecimentoHospedagem,23,kendall.faria@gmail.com,Avenida Avenida Boavista 911,4100-128,"União das freguesias de Cedofeita, Santo Ildef...",Porto,Porto
1,293,-8.602860,41.144624,185476726,14,Centenry Fontaínhas Apartments 1,2014/12/01 09:40:52+00,2014/12/01 00:00:00+00,Apartamento,6,apcf.pc@gmail.com,Rua Fontaínhas 167 1.º,4000-240,"União das freguesias de Cedofeita, Santo Ildef...",Porto,Porto
2,294,-8.602860,41.144624,185476727,15,Centenry Fontaínhas Apartments 1,2014/12/01 09:56:17+00,2014/12/01 00:00:00+00,Apartamento,6,apcf.pc@gmail.com,Rua Fontaínhas 167,4000-240,"União das freguesias de Cedofeita, Santo Ildef...",Porto,Porto
3,295,-8.602860,41.144624,185476728,16,Centenry Fontaínhas Apartments 3,2014/12/01 10:04:00+00,2014/12/01 00:00:00+00,Apartamento,6,apcf.pc@gmail.com,Rua Fontaínhas 167 3.º,4000-240,"União das freguesias de Cedofeita, Santo Ildef...",Porto,Porto
4,296,-8.614323,41.142069,185476729,32,inPátio Guest House,2014/12/01 16:41:53+00,2012/09/10 00:00:00+00,EstabelecimentoHospedagem,12,info@inpatio.pt,Pátio São Salvador 22,4050-567,"União das freguesias de Cedofeita, Santo Ildef...",Porto,Porto


In [5]:
input_file = "BGRI/BGRI.gpkg"
BGRI = gpd.read_file(input_file)

BGRI.head()

,OBJECTID,BGRI2021,DT21,DTMN21,DTMNFR21,DTMNFRSEC21,SECNUM21,SSNUM21,SECSSNUM21,SUBSECCAO,...,N_INDIVIDUOS,N_INDIVIDUOS_H,N_INDIVIDUOS_M,N_INDIVIDUOS_0_14,N_INDIVIDUOS_15_24,N_INDIVIDUOS_25_64,N_INDIVIDUOS_65_OU_MAIS,SHAPE_Length,SHAPE_Area,geometry
0,52,13121000702,13,1312,131210,131210007,007,02,00702,13121000702,...,328.0,145.0,183.0,29.0,34.0,184.0,81.0,612.308514,21064.473451,"MULTIPOLYGON (((-40792.851 166989.435, -40733...."
1,53,13120302118,13,1312,131203,131203021,021,18,02118,13120302118,...,134.0,54.0,80.0,8.0,10.0,55.0,61.0,481.233389,13437.276410,"MULTIPOLYGON (((-37680.254 167001.264, -37667...."
2,54,13121103402,13,1312,131211,131211034,034,02,03402,13121103402,...,143.0,67.0,76.0,15.0,8.0,79.0,41.0,707.178859,13484.048739,"MULTIPOLYGON (((-43406.159 166894.230, -43347...."
3,55,13121600903,13,1312,131216,131216009,009,03,00903,13121600903,...,205.0,89.0,116.0,33.0,12.0,100.0,60.0,824.199641,24498.620921,"MULTIPOLYGON (((-45059.181 166999.619, -45056...."
4,56,13121600403,13,1312,131216,131216004,004,03,00403,13121600403,...,5.0,2.0,3.0,0.0,1.0,2.0,2.0,177.035284,1829.033862,"MULTIPOLYGON (((-43954.343 166993.913, -43965...."


In [16]:
BGRI = BGRI.to_crs("EPSG:4326")
points_gdf = gpd.GeoDataFrame(RNAL, geometry=gpd.points_from_xy(RNAL['X'], RNAL['Y']), crs="EPSG:4326")
points_in_areas = gpd.sjoin(points_gdf, BGRI, how='left', op='within')

# Count the points in each area
point_counts = points_in_areas.groupby('BGRI2021').size().reset_index(name='point_count')


        BGRI2021  point_count
0    13120200101           21
1    13120200104           28
2    13120200105            1
3    13120200106            3
4    13120200107           34
..           ...          ...
809  13121802505            1
810  13121802506            5
811  13121802510            4
812  13121802511            8
813  13121802512            1

[814 rows x 2 columns]


/opt/homebrew/opt/ipython/libexec/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [19]:
point_counts["point_count"].value_counts()

point_count
1     209
2     108
3      67
4      46
5      43
     ... 
41      1
57      1
74      1
93      1
50      1
Name: count, Length: 84, dtype: int64

In [20]:
merged_df = pd.merge(BGRI, point_counts, on='BGRI2021', how='left')

# Fill NaN values with 0 in the 'number_of_inhabitants' column
merged_df['point_count'] = merged_df['point_count'].fillna(0)

In [23]:
merged_df = merged_df.rename(columns={"point_count":"ALs"})

In [33]:
merged_df["pressao"] = merged_df["ALs"] / merged_df["N_ALOJAMENTOS_FAM_CLASS_RHABITUAL"]

In [32]:
merged_df["N_ALOJAMENTOS_FAMILIARES"].value_counts()

N_ALOJAMENTOS_FAM_CLASS_RHABITUAL
0.0      63
12.0     39
9.0      33
6.0      33
8.0      29
         ..
233.0     1
179.0     1
296.0     1
154.0     1
175.0     1
Name: count, Length: 257, dtype: int64

In [34]:
merged_df.to_csv("heatmap.csv")

In [35]:
merged_df.columns
merged_df = merged_df.drop("index_right", axis = 1)


Index(['OBJECTID', 'BGRI2021', 'DT21', 'DTMN21', 'DTMNFR21', 'DTMNFRSEC21',
       'SECNUM21', 'SSNUM21', 'SECSSNUM21', 'SUBSECCAO', 'NUTS1', 'NUTS2',
       'NUTS3', 'N_EDIFICIOS_CLASSICOS', 'N_EDIFICIOS_CLASS_CONST_1_OU_2_ALOJ',
       'N_EDIFICIOS_CLASS_CONST_3_OU_MAIS_ALOJAMENTOS',
       'N_EDIFICIOS_EXCLUSIV_RESID', 'N_EDIFICIOS_1_OU_2_PISOS',
       'N_EDIFICIOS_3_OU_MAIS_PISOS', 'N_EDIFICIOS_CONSTR_ANTES_1945',
       'N_EDIFICIOS_CONSTR_1946_1980', 'N_EDIFICIOS_CONSTR_1981_2000',
       'N_EDIFICIOS_CONSTR_2001_2010', 'N_EDIFICIOS_CONSTR_2011_2021',
       'N_EDIFICIOS_COM_NECESSIDADES_REPARACAO', 'N_ALOJAMENTOS_TOTAL',
       'N_ALOJAMENTOS_FAMILIARES', 'N_ALOJAMENTOS_FAM_CLASS_RHABITUAL',
       'N_ALOJAMENTOS_FAM_CLASS_VAGOS_OU_RESID_SECUNDARIA',
       'N_RHABITUAL_ACESSIVEL_CADEIRAS_RODAS',
       'N_RHABITUAL_COM_ESTACIONAMENTO', 'N_RHABITUAL_PROP_OCUP',
       'N_RHABITUAL_ARRENDADOS', 'N_AGREGADOS_DOMESTICOS_PRIVADOS',
       'N_ADP_1_OU_2_PESSOAS', 'N_ADP_3_OU_MAIS_PE

In [62]:
input_file = "CAOP/CAOP.gpkg"
CAOP = gpd.read_file(input_file)
CAOP = CAOP[CAOP["Concelho"] == "Porto"][["Freguesia", "geometry"]]
CAOP = CAOP.to_crs("EPSG:4326")
CAOP.head()

,Freguesia,geometry
928,Ramalde,"MULTIPOLYGON (((-8.62268 41.18025, -8.62278 41..."
932,Paranhos,"MULTIPOLYGON (((-8.59688 41.18375, -8.59683 41..."
2229,União das freguesias de Lordelo do Ouro e Mass...,"MULTIPOLYGON (((-8.63594 41.15976, -8.63587 41..."
2233,Bonfim,"MULTIPOLYGON (((-8.58820 41.16705, -8.58839 41..."
2234,"União das freguesias de Cedofeita, Santo Ildef...","MULTIPOLYGON (((-8.60412 41.16182, -8.60400 41..."


In [63]:
intersections = gpd.overlay(merged_df, CAOP, how='intersection')
# Calculate the area of each intersection
intersections['intersection_area'] = intersections['geometry'].area

# Find the county with the maximum intersection area for each small section
idx = intersections.groupby('BGRI2021')['intersection_area'].idxmax()

# Create a new DataFrame with only the small section name and county name
result_df = intersections.loc[idx, ['BGRI2021', 'Freguesia']]

# Display the result
print(result_df)

         BGRI2021                                          Freguesia
1726  13120200101                                             Bonfim
1732  13120200102                                             Bonfim
1687  13120200103                                             Bonfim
1688  13120200104                                             Bonfim
1734  13120200105                                             Bonfim
...           ...                                                ...
1531  13121802508  União das freguesias de Lordelo do Ouro e Mass...
1653  13121802509  União das freguesias de Lordelo do Ouro e Mass...
1525  13121802510  União das freguesias de Lordelo do Ouro e Mass...
1533  13121802511  União das freguesias de Lordelo do Ouro e Mass...
1512  13121802512  União das freguesias de Lordelo do Ouro e Mass...

[1659 rows x 2 columns]


/opt/homebrew/lib/python3.11/site-packages/geopandas/geodataframe.py:1815: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  merged_geom = block.unary_union
/var/folders/2_/dc69y37n7d14q5rbg8ly14g40000gn/T/ipykernel_38402/1137887932.py:1: UserWarning: `keep_geom_type=True` in overlay resulted in 387 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  intersections = gpd.overlay(merged_df, CAOP, how='intersection')
/var/folders/2_/dc69y37n7d14q5rbg8ly14g40000gn/T/ipykernel_38402/1137887932.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  intersections['intersection_area'] = intersections['geometry'].area


In [83]:
result_df["BGRI2021"] = result_df["BGRI2021"].astype(int).reindex("BGRI2021")
merged_df["BGRI2021"] = merged_df["BGRI2021"].astype(int).set_index("BGRI2021")

TypeError: Index(...) must be called with a collection of some kind, 'BGRI2021' was passed

In [87]:
merged_df = merged_df.join(result_df, on="BGRI2021", how="left")

In [86]:
result_df = result_df.set_index("BGRI2021")

In [89]:
merged_df.to_csv("seccoes.csv")

In [95]:
merged_df.groupby("Freguesia")["N_ALOJAMENTOS_FAMILIARES"].agg("sum")

Freguesia
Bonfim                                                                                     14802.0
Campanhã                                                                                   16022.0
Paranhos                                                                                   27439.0
Ramalde                                                                                    19126.0
União das freguesias de Aldoar, Foz do Douro e Nevogilde                                   14054.0
União das freguesias de Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau e Vitória    26979.0
União das freguesias de Lordelo do Ouro e Massarelos                                       14939.0
Name: N_ALOJAMENTOS_FAMILIARES, dtype: float64

In [96]:
merged_df.groupby("Freguesia")["ALs"].agg("sum") / merged_df.groupby("Freguesia")["N_ALOJAMENTOS_FAMILIARES"].agg("sum")

Freguesia
Bonfim                                                                                     0.095798
Campanhã                                                                                   0.013107
Paranhos                                                                                   0.013375
Ramalde                                                                                    0.008000
União das freguesias de Aldoar, Foz do Douro e Nevogilde                                   0.017504
União das freguesias de Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau e Vitória    0.279551
União das freguesias de Lordelo do Ouro e Massarelos                                       0.035277
dtype: float64

In [121]:
input_file = "CAOP/CAOP2011.shp"
CAOP11 = gpd.read_file(input_file)
CAOP11 = CAOP11[CAOP11["MUNICIPIO"] == "PORTO"][["FREGUESIA", "geometry"]]
CAOP11 = CAOP11.to_crs("EPSG:4326")
CAOP11.head()

,FREGUESIA,geometry
2125,CAMPANHÃ,"POLYGON ((-8.56821 41.17755, -8.56783 41.17728..."
2126,ALDOAR,"POLYGON ((-8.66185 41.17696, -8.66154 41.17692..."
2135,RAMALDE,"POLYGON ((-8.63266 41.17926, -8.63240 41.17921..."
2141,PARANHOS,"POLYGON ((-8.60353 41.18586, -8.60237 41.18526..."
2983,MASSARELOS,"POLYGON ((-8.63565 41.15900, -8.63327 41.15861..."


In [122]:
intersections = gpd.overlay(merged_df, CAOP, how='intersection')
# Calculate the area of each intersection
intersections['intersection_area'] = intersections['geometry'].area

# Find the county with the maximum intersection area for each small section
idx = intersections.groupby('BGRI2021')['intersection_area'].idxmax()

# Create a new DataFrame with only the small section name and county name
result_df = intersections.loc[idx, ['BGRI2021', 'FREGUESIA']]

# Display the result
print(result_df)

         BGRI2021        FREGUESIA
1217  13120200101           BONFIM
1223  13120200102           BONFIM
1175  13120200103           BONFIM
1176  13120200104           BONFIM
1225  13120200105           BONFIM
...           ...              ...
1620  13121802508  LORDELO DO OURO
1701  13121802509  LORDELO DO OURO
1617  13121802510  LORDELO DO OURO
1621  13121802511  LORDELO DO OURO
1608  13121802512  LORDELO DO OURO

[1659 rows x 2 columns]


/opt/homebrew/lib/python3.11/site-packages/geopandas/geodataframe.py:1815: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  merged_geom = block.unary_union
/var/folders/2_/dc69y37n7d14q5rbg8ly14g40000gn/T/ipykernel_38402/1373537385.py:1: UserWarning: `keep_geom_type=True` in overlay resulted in 449 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  intersections = gpd.overlay(merged_df, CAOP, how='intersection')
/var/folders/2_/dc69y37n7d14q5rbg8ly14g40000gn/T/ipykernel_38402/1373537385.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  intersections['intersection_area'] = intersections['geometry'].area


In [123]:
result_df["BGRI2021"] = result_df["BGRI2021"].astype(int).reindex("BGRI2021")
merged_df["BGRI2021"] = merged_df["BGRI2021"].astype(int).set_index("BGRI2021")

TypeError: Index(...) must be called with a collection of some kind, 'BGRI2021' was passed

In [124]:
result_df = result_df.set_index("BGRI2021")

In [126]:
result_df = result_df.rename(columns={"FREGUESIA":"Freguesia2011"})

In [129]:
merged_df = merged_df.join(result_df, on="BGRI2021", how="left")

In [132]:
merged_df.to_csv("seccoes-final.csv")

In [133]:
merged_df.groupby("Freguesia2011")["ALs"].agg("sum") / merged_df.groupby("Freguesia2011")["N_ALOJAMENTOS_FAMILIARES"].agg("sum") * 100

Freguesia2011
ALDOAR              0.499500
BONFIM              9.579787
CAMPANHÃ            1.310698
CEDOFEITA          11.604575
FOZ DO DOURO        3.370787
LORDELO DO OURO     1.314918
MASSARELOS          8.882784
MIRAGAIA           26.466165
NEVOGILDE           1.260023
PARANHOS            1.337512
RAMALDE             0.799958
SANTO ILDEFONSO    42.279185
SÃO NICOLAU        58.264463
SÉ                 57.345491
VITÓRIA            79.765625
dtype: float64